In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
import geopandas as gpd
import xarray as xr
from shapely.geometry import mapping
shp_pan = gpd.read_file(r'Data\shapefiles\panamz.geojson').to_crs('EPSG:4326')


In [2]:
##clip cru to amz shp
#from shapely.geometry import mapping
folder_clean = r'Data\Datasets\clean'
folder_amz = r'Data\Datasets\amz\ts'
#filter from 2001 to 2020
begin_date = '2001-01-01'
end_date = '2020-12-31'
lat_min = -20
lat_max = 11
lon_min = -81
lon_max = -42


In [2]:
datasets = ['cru', 'gpcc', 'chirps','imerg', 'terra', 'era_land', 'jra55','merra2']
datasets_names = ['CRU', 'GPCC', 'CHIRPS','IMERG-V6', 'TerraClimate', 'ERA5-Land', 'JRA55','MERRA2']

In [3]:
for dataset in datasets:
    #read file of dataset in folder_clean
    file_path = glob(os.path.join(folder_clean, dataset + '.nc'))
    ds = xr.open_dataset(file_path[0]).sel(time=slice(begin_date, end_date), lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
    ds_amz = ds.rio.write_crs(shp_pan.crs).rio.set_spatial_dims(x_dim='lon', y_dim='lat').rio.clip(shp_pan.geometry.apply(mapping), shp_pan.crs, drop=False)
    #save clipped dataset
    if dataset == 'terra':
        vars_list = list(ds_amz.data_vars)  
        for var in vars_list:  
            del ds_amz[var].attrs['grid_mapping']


    ds_amz.to_netcdf(os.path.join(folder_amz, dataset + '.nc'))

NameError: name 'glob' is not defined

In [3]:
datasets_extras = ['cpc', 'cmap', 'gpcp', 'persiann', 'ccs', 'cdr', 'mswep',
                    'mswep_nogauge', 'gldas', 'era', 'ncep1', 'ncep2']
datasets_names_extras = ['CPC', 'CMAP', 'GPCP v3.2', 'PERSIANN', 'PERSIANN-CCS', 'PERSIANN-CDR', 'MSWEP v2.8',
                    'MSWEP_nogauge v2.8', 'GLDAS v2.1', 'ERA5', 'NCEP R1', 'NCEP R2']


for dataset in datasets_extras:
    #read file of dataset in folder_clean
    file_path = glob(os.path.join(folder_clean, dataset + '.nc'))
    ds = xr.open_dataset(file_path[0]).sel(time=slice(begin_date, end_date), lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
    ds_amz = ds.pr.rio.write_crs('EPSG:4326').rio.set_spatial_dims(x_dim='lon', y_dim='lat').rio.clip(shp_pan.geometry.apply(mapping),  crs=shp_pan.crs,drop=False)
    #save clipped dataset
    ds_amz = ds_amz.to_dataset(name='pr')
    #delete grid mapping if it exists
    if 'grid_mapping' in ds_amz.pr.attrs:
        del ds_amz.pr.attrs['grid_mapping']

    print(dataset)
    ds_amz.to_netcdf(os.path.join(folder_amz, dataset + '.nc'))

ccs
